In [1]:
from np_struct import Packet, Struct
import numpy as np

# Numpy Structures

`np-structures` extends structured arrays in NumPy to be a bit more user friendly and intuitive, with added support for transferring structured arrays across serial or socket interfaces. 
 
Structured arrays are built to mirror the struct typedef in C/C++, but can be used for any complicated data structure. They behave similar to standard arrays, but support mixed data types, labeling, and unequal length arrays. Arrays are easily written or loaded from disk in the standard `.npy` binary format.

## Creating Structs

Struct members are listed as class variables, and can be other Structs, numpy arrays, or one of the standard numpy data types.

In [2]:
class pktheader(Struct):
    psize = np.uint16()
    dest =  np.uint8()
    src =   np.uint8()
    ptype = np.uint8()

When creating a new struct object from a declared type, any member can be initialized by passing in it's value with a kwarg.

In [3]:
pkt = pktheader(ptype=5)
pkt

Struct pktheader: 
    psize:  uint16[0]
    dest:   uint8[0]
    src:    uint8[0]
    ptype:  uint8[5]

An array of structs can be created by passing in the `shape` kwarg. 

In [4]:
pkt_array = pktheader(ptype=5, shape=(3,2))
pkt_array

Struct pktheader: (3, 2)
[
    psize:  uint16[0]
    dest:   uint8[0]
    src:    uint8[0]
    ptype:  uint8[5]
]
...
[
    psize:  uint16[0]
    dest:   uint8[0]
    src:    uint8[0]
    ptype:  uint8[5]
]

Initialized members will be broadcasted to the shape of the initial value.

In [5]:
pkt = pktheader(ptype=np.arange(5))
pkt

Struct pktheader: 
    psize:  uint16[0]
    dest:   uint8[0]
    src:    uint8[0]
    ptype:  uint8[0 1 2 3 4]

Structures can also include other structures.

In [6]:
class expkt(Struct):
    hdr = pktheader(ptype=0x2)
    da = np.arange(8)

ex = expkt()
ex

Struct expkt: 
    hdr:  Struct pktheader: 
            psize:  uint16[0]
            dest:   uint8[0]
            src:    uint8[0]
            ptype:  uint8[2]
    da:   int32[0 1 2 3 4 5 6 7]

## Accessing and Setting Members

Members can be accessed either with indexing, or using the dot operator.

In [7]:
ex.da *= 2
ex['da']

array([ 0,  2,  4,  6,  8, 10, 12, 14])

In [8]:
pkt_array[0,0].ptype = 15
pkt_array

Struct pktheader: (3, 2)
[
    psize:  uint16[0]
    dest:   uint8[0]
    src:    uint8[0]
    ptype:  uint8[15]
]
...
[
    psize:  uint16[0]
    dest:   uint8[0]
    src:    uint8[0]
    ptype:  uint8[5]
]

## Writing to Disk

In [9]:
np.save('test.npy', pkt_array)
pktheader(np.load('test.npy'))

Struct pktheader: (3, 2)
[
    psize:  uint16[0]
    dest:   uint8[0]
    src:    uint8[0]
    ptype:  uint8[15]
]
...
[
    psize:  uint16[0]
    dest:   uint8[0]
    src:    uint8[0]
    ptype:  uint8[5]
]

## Interfaces

To send structured arrays across a serial or socket interface, the structure must include a member that holds the structure size in bytes, and another that holds a type field that is unique for each structure class that will be sent across the interface. 

To register the structure with the interface, the structure must inherit from a base class that includes the size and type fields, and implements four methods shown below.

In [10]:
from np_struct.fields import uint16

class BasePacket(Packet):
    hdr = pktheader()

    def set_psize(self, value):
        self.hdr.psize = value

    def set_ptype(self, value):
        self.hdr.ptype = value
    
    def get_ptype(self):
        return self.hdr.ptype
    
    def get_psize(self):
        return self.hdr.psize

class datapkt(BasePacket):
    hdr = pktheader(ptype=0x2)
    da = np.zeros(10)

class testpkt(BasePacket):
    hdr = pktheader(ptype=0x03)

class ack(BasePacket):
    hdr = pktheader(ptype=0xFF)
    ack_type = np.uint8()
    ack_code = np.uint8()

The `datapkt` and `testpkt` structures will be recognized by an interface since they inherit from `BasePacket`.

To implement an interface that recognizes these structures, pass the `BasePacket` class into the `pkt_class` kwarg.

In [11]:
from np_struct.transfer import SocketInterface
import threading

class SimpleServer(threading.Thread):

    def __init__(self, intf):
        super().__init__()
        self.terminate_flag = threading.Event()
        self.intf = intf

    def stop(self):
        self.terminate_flag.set()

    def run(self):

        while not self.terminate_flag.is_set():
            # accept connections from clients until the terminate flag is set
            try:
                self.intf.connect()
            except TimeoutError:
                self.intf.close()
                continue
            
            # if connection made, read packet from socket. Packet may be any type that inherits from BasePacket
            pkt = self.intf.pkt_read()

            # fill data field with random integers and send back to client
            if isinstance(pkt, datapkt):
                pkt.da = np.random.randint(0, 0xFF, size=pkt.da.shape)
                self.intf.pkt_write(pkt)
            # send acknowledgment back to client
            else:
                ackpkt = ack()
                ackpkt.ack_type = pkt.hdr.ptype
                self.intf.pkt_write(ackpkt)
            
            self.intf.close()

# create server interface by providing a host port to bind to
server_intf = SocketInterface(host=('localhost', 50007), pkt_class=BasePacket, timeout=2)
# create client by providing a target port to connect to
client_intf = SocketInterface(target=('localhost', 50007), pkt_class=BasePacket)

# start server thread
server = SimpleServer(server_intf)
server.start()

In [12]:
with client_intf as client:
    ex = datapkt()
    rxpkt = client.pkt_sendrecv(ex)

    print(rxpkt)


BasePacket datapkt: 
    hdr:  Struct pktheader: 
            psize:  uint16[85]
            dest:   uint8[0]
            src:    uint8[0]
            ptype:  uint8[2]
    da:   float64[250. 179.  15. 183.  90. 132. 221. 186. 239. 250.]


In [13]:
with client_intf as client:

    ex = testpkt()
    rxpkt = client.pkt_sendrecv(ex)

    print(rxpkt)


BasePacket ack: 
    hdr:       Struct pktheader: 
            psize:  uint16[7]
            dest:   uint8[0]
            src:    uint8[0]
            ptype:  uint8[255]
    ack_type:  uint8[3]
    ack_code:  uint8[0]


In [14]:
server.stop()